In [ ]:
%pip install faiss-cpu==1.7.4 chromadb==0.3.21

Note: you may need to restart the kernel using dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 64.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 72.3 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 964.5/964.5 kB 80.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 91.8 MB/s eta 0:00:00
   


### Search Semantics

In [ ]:
from datasets import load_dataset
from transformers import pipeline
import pandas as pd
import numpy as np

In [ ]:
xsum_dataset = load_dataset(
    "xsum", version="1.2.0"
)  

/databricks/python_shell/dbruntime/huggingface_patches/datasets.py:27: UserWarning: This dataset can not be stored in DBFS because either `cache_dir` or the environment variable `HF_DATASETS_CACHE` is set to a non-DBFS path. If this cluster restarts, all saved dataset information will be lost.
  warnings.warn(


/databricks/python_shell/dbruntime/huggingface_patches/datasets.py:13: UserWarning: During large dataset downloads, there could be multiple progress bar widgets that can cause performance issues for your notebook or browser. To avoid these issues, use `datasets.utils.logging.disable_progress_bar()` to turn off the progress bars.
  warnings.warn(


Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

Dataset xsum downloaded and prepared to /root/.cache/huggingface/datasets/xsum/default/1.2.0/082863bf4754ee058a5b6f6525d0cb2b18eadb62c7b370b095d1364050a52b71. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# Taking a sample of 100 rows
xsum_sample = xsum_dataset["train"].select(range(1000)).to_pandas()

# Combining 'document' and 'summary' columns
xsum_sample["combined"] = (
    "Document: " + xsum_sample.document.str.strip() + "; Summary: " + xsum_sample.summary.str.strip()
)

In [ ]:
from sentence_transformers import SentenceTransformer

#encoding the data
encoder = SentenceTransformer("paraphrase-mpnet-base-v2")
encoded_data = encoder.encode(xsum_sample["combined"])

In [ ]:
import faiss

In [ ]:
index = faiss.IndexIDMap(faiss.IndexFlatIP(768))

In [ ]:
faiss.normalize_L2(encoded_data)
index.add_with_ids(encoded_data, np.arange(len(encoded_data)))

In [ ]:
search_text = "harry potter"

In [ ]:
search_vector = encoder.encode(search_text)
_vector = np.array([search_vector])
faiss.normalize_L2(_vector)

k = 2
distances, ann = index.search(_vector, k=k)
results = pd.DataFrame({'distances': distances[0], 'ann': ann[0]})

In [ ]:
print(results)

   distances  ann
0   0.598044  269
1   0.379589  514


In [ ]:
xsum_sample["summary"][results['ann'][0]]

'Harry Potter and the Cursed Child has won five-star reviews from critics, with one describing it as "a game-changing production".'

In [ ]:
xsum_sample["document"][results['ann'][0]]

'The play, written by Jack Thorne, is set 19 years after the seventh and final book in the series by JK Rowling.\nIt opens officially at the Palace Theatre, in London\'s West End, on Saturday.\nAudiences have been urged to "keep the secrets" since the play began previews in early June.\nPresented in two parts, the play - showing the stars of the wizarding saga as adults in their mid-30s as their own children head off to school - stretches over five hours.\nDaily Telegraph critic Dominic Cavendish awarded the play five stars, writing: "British theatre hasn\'t known anything like it for decades and I haven\'t seen anything directly comparable in all my reviewing days."\nHe said "those involved can give themselves a pat on the back", adding: "It\'s a triumph. Not an unqualified one - there are some quibbles - but in all key respects, it grips, it stirs, it delights."\nCavendish praises the "thrill-a-minute" stage craft which sees pupils heading to Hogwarts, at the start of the play, chang

In [ ]:
xsum_sample["summary"][results['ann'][1]]

'Author Patrick Ness is in the running to become the first author to win the Carnegie Medal three times.'

In [ ]:
xsum_sample["document"][results['ann'][1]]

'He has been shortlisted for the children\'s book award for The Rest of Us Just Live Here, which follows the lives of a group of teenagers.\nFrancis Hardinge, who won the Costa Book of the Year award for The Lie Tree, is also shortlisted.\nThree illustrators are also in with a chance of winning the Kate Greenaway Medal for a record third time.\nChildren\'s Laureate Chris Riddell, former Children\'s Laureate Anthony Browne and Helen Oxenbury - who first won the award for outstanding illustration in a book for children in 1969 - have all made the shortlist, which is dominated by traditional picture books this year.\nBrowne first won the medal in 1983, while Riddell won for the first time in 2001.\nHardinge\'s Victorian murder mystery joins Lies We Tell Ourselves, author Robin Talley\'s first book which tells of two teenage girls falling in love across the race divide in 1950s America.\nSarah Crossan\'s One, about conjoined twins; Marcus Sedgwick\'s The Ghosts of Heaven, which looks at th


### Summarization

In [ ]:
summarizer = pipeline(
    task="summarization",
    model="t5-small",
    min_length=20,
    max_length=60,
    truncation=True,
) 

In [ ]:
xsum_sample["summary"][results['ann'][0]]

'Harry Potter and the Cursed Child has won five-star reviews from critics, with one describing it as "a game-changing production".'

In [ ]:
summarizer(xsum_sample["document"][results['ann'][0]])

[{'summary_text': 'the play, written by Jack Thorne, is set 19 years after the seventh and final book in the series by JK Rowling . it opens officially at the palace theatre, in London\'s west end, on Saturday . critics have been urged to "keep the'}]

In [ ]:
# We can instead do a beam search by specifying num_beams.
summarizer(xsum_sample["document"][results['ann'][0]], num_beams=10)

[{'summary_text': 'the play, written by Jack Thorne, is set 19 years after the seventh book in the series by JK Rowling . it opens officially at the palace theatre, in London\'s west end, on sunday . critics have been urged to "keep the secrets'}]

In [ ]:
# Alternatively, we could use sampling.
summarizer(xsum_sample["document"][results['ann'][0]], do_sample=True)

[{'summary_text': 'the play, written by Jack Thorne, is set 19 years after the seventh and final book in the series by JK Rowling . it opens officially at the palace theatre, in London\'s west end, on Saturday . critics have been urged to "keep the'}]

In [ ]:
# We can modify sampling to be more greedy by limiting sampling to the top_k or top_p most likely next tokens.
summarizer(xsum_sample["document"][results['ann'][0]], do_sample=True, top_k=10, top_p=0.8)

[{'summary_text': 'the play, written by Jack Thorne, opens at the palace theatre on saturday . it shows the stars of the wizarding saga as adults in their mid-30s as their own children head off to school . critics have been urged to "keep'}]